### New project

build configuration language : Groovy DSL 로 변경

### build.gradle

```
dependencies {
    ...

    implementation 'org.tensorflow:tensorflow-lite:2.7.0'
    implementation 'org.tensorflow:tensorflow-lite-support:0.3.0'
}
```

### Sync Now

### app > java > com.example.iris > Classifier

```Classifier

package com.example.iris;

import android.content.Context;
import android.content.res.AssetFileDescriptor;
import android.content.res.AssetManager;
import android.graphics.Bitmap;
import android.util.Log;
import android.util.Pair;
import org.tensorflow.lite.DataType;
import org.tensorflow.lite.Interpreter;
import org.tensorflow.lite.Tensor;
import org.tensorflow.lite.support.tensorbuffer.TensorBuffer;
import java.io.FileInputStream;
import java.io.IOException;
import java.nio.ByteBuffer;
import java.nio.ByteOrder;
import java.nio.channels.FileChannel;
import java.util.ArrayList;
import java.util.List;

public class Classifier {
    private static final String MODEL_NAME = "iris_model.tflite";
    Context context;
    Interpreter interpreter = null;
    int modelOutputClasses;
    public Classifier(Context context) {
        this.context = context;
    }
    public void init() throws IOException {
        ByteBuffer model = null;
        model = loadModelFile(MODEL_NAME);
        model.order(ByteOrder.nativeOrder());
        interpreter = new Interpreter(model);
        initModelShape();
    }
    private ByteBuffer loadModelFile(String modelName) throws IOException {
        AssetManager am = context.getAssets();
        AssetFileDescriptor afd = am.openFd(modelName);
        FileInputStream fis = new FileInputStream(afd.getFileDescriptor());
        FileChannel fc = fis.getChannel();
        long startOffset = afd.getStartOffset();
        long declaredLength = afd.getDeclaredLength();
        return fc.map(FileChannel.MapMode.READ_ONLY, startOffset, declaredLength);
    }
    private void initModelShape() {
        Tensor inputTensor = interpreter.getInputTensor(0);
        Tensor outputTensor = interpreter.getOutputTensor(0);
        int[] outputShape = outputTensor.shape();
        modelOutputClasses = outputShape[1];
    }
    public List<Object> classify(float input1, float input2, float input3, float input4) {
        float[] inputData = {input1, input2, input3, input4};
        float[][] outputData = new float[1][3];
        interpreter.run(inputData, outputData);
        int argmax = 0;
        float max = outputData[0][0];
        for (int i = 1; i < outputData.length; i++) {
            float f = outputData[0][i];
            if (f > max) {
                argmax = i;
                max = f;
            }
        }
        List<Object> items = new ArrayList<>();
        items.add(argmax);
        items.add(max);
        return items;
    }
    public void finish() {
        if (interpreter != null)
            interpreter.close();
    }
}

```

com.example.iris 우클릭 new > activity> empty > launcher activity 선택
